In [1]:
## This file implements neural networks and random forest on p0006presabsSTCC_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 64.96% for over-sampling data and 72.65%  after improvement.
## And it is 83.87% for combination data and 96.77% after regularization and improvement.
## For random forest, the accuracy is 69.23% for over-sampling data and 100% for combination data.
## For random forest with cross-validation, the mean accuracy is 68.91% for over-sampling, and 89.14% for combination data.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0006presabsSTCC_qual.csv')
df.shape

(255, 88)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      1
1      0
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      1
10     0
11     2
12     1
13     0
14     0
15     0
16     1
17     1
18     2
19     0
20     1
21     1
22     0
23     0
24     2
25     0
26     2
27     0
28     0
29     2
      ..
225    1
226    1
227    1
228    2
229    2
230    1
231    0
232    1
233    1
234    1
235    2
236    0
237    0
238    0
239    0
240    0
241    1
242    0
243    0
244    1
245    0
246    1
247    0
248    0
249    0
250    1
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [5]:
df['pheno'].value_counts()

0    130
1     86
2     39
Name: pheno, dtype: int64

In [6]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,...,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,ST,CC,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1


In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(255, 87)

In [9]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,...,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,ST,CC,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1


In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 86) (255,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 130), (1, 130), (2, 130)]


Using TensorFlow backend.


In [12]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 27), (1, 11), (2, 64)]


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [14]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [15]:
############# Fully-Connected Neural Network ################

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [17]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 273 samples, validate on 117 samples
Epoch 1/100
273/273 [==============================] - 0s 400us/step - loss: 20.0644 - accuracy: 0.3700 - val_loss: 24.9884 - val_accuracy: 0.4188
Epoch 2/100
273/273 [==============================] - 0s 92us/step - loss: 15.4777 - accuracy: 0.3700 - val_loss: 19.3247 - val_accuracy: 0.3846
Epoch 3/100
273/273 [==============================] - 0s 125us/step - loss: 11.8476 - accuracy: 0.3773 - val_loss: 13.1560 - val_accuracy: 0.4017
Epoch 4/100
273/273 [==============================] - 0s 110us/step - loss: 7.7628 - accuracy: 0.4139 - val_loss: 7.4550 - val_accuracy: 0.3761
Epoch 5/100
273/273 [==============================] - 0s 103us/step - loss: 4.2965 - accuracy: 0.4396 - val_loss: 4.7373 - val_accuracy: 0.3761
Epoch 6/100
273/273 [==============================] - 0s 103us/step - loss: 3.3549 - accuracy: 0.4103 - val_loss: 3.7068 - val_accuracy: 0.3846
Epoch 7/100
273/273 [==============================] - 0s 100us/step - loss: 2.

Epoch 57/100
273/273 [==============================] - 0s 108us/step - loss: 0.7054 - accuracy: 0.7326 - val_loss: 0.9615 - val_accuracy: 0.6068
Epoch 58/100
273/273 [==============================] - 0s 88us/step - loss: 0.7113 - accuracy: 0.7399 - val_loss: 0.8564 - val_accuracy: 0.6239
Epoch 59/100
273/273 [==============================] - 0s 81us/step - loss: 0.6789 - accuracy: 0.7363 - val_loss: 0.8971 - val_accuracy: 0.5897
Epoch 60/100
273/273 [==============================] - 0s 78us/step - loss: 0.6693 - accuracy: 0.7436 - val_loss: 0.9465 - val_accuracy: 0.5983
Epoch 61/100
273/273 [==============================] - 0s 88us/step - loss: 0.7050 - accuracy: 0.7363 - val_loss: 0.8719 - val_accuracy: 0.6239
Epoch 62/100
273/273 [==============================] - 0s 75us/step - loss: 0.6317 - accuracy: 0.7656 - val_loss: 0.8945 - val_accuracy: 0.6325
Epoch 63/100
273/273 [==============================] - 0s 76us/step - loss: 0.6501 - accuracy: 0.7399 - val_loss: 0.8486 - val_a

In [20]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

117/117 [==============================] - 0s 146us/step
over-sampling test accuracy: 64.96%


In [72]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [73]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [74]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 273 samples, validate on 117 samples
Epoch 1/1000
273/273 [==============================] - 0s 445us/step - loss: 9.5235 - accuracy: 0.2747 - val_loss: 8.4119 - val_accuracy: 0.3419
Epoch 2/1000
273/273 [==============================] - 0s 62us/step - loss: 4.5000 - accuracy: 0.2967 - val_loss: 2.4934 - val_accuracy: 0.2906
Epoch 3/1000
273/273 [==============================] - 0s 77us/step - loss: 1.7461 - accuracy: 0.3150 - val_loss: 3.0498 - val_accuracy: 0.3419
Epoch 4/1000
273/273 [==============================] - 0s 74us/step - loss: 2.6438 - accuracy: 0.3407 - val_loss: 3.5502 - val_accuracy: 0.3248
Epoch 5/1000
273/273 [==============================] - 0s 79us/step - loss: 2.5289 - accuracy: 0.3407 - val_loss: 2.2453 - val_accuracy: 0.3248
Epoch 6/1000
273/273 [==============================] - 0s 89us/step - loss: 1.6016 - accuracy: 0.3480 - val_loss: 1.9919 - val_accuracy: 0.2735
Epoch 7/1000
273/273 [==============================] - 0s 77us/step - loss: 1.6088

Epoch 57/1000
273/273 [==============================] - 0s 93us/step - loss: 1.0980 - accuracy: 0.6740 - val_loss: 1.1723 - val_accuracy: 0.5128
Epoch 58/1000
273/273 [==============================] - 0s 68us/step - loss: 0.8248 - accuracy: 0.6593 - val_loss: 1.1003 - val_accuracy: 0.5128
Epoch 59/1000
273/273 [==============================] - 0s 73us/step - loss: 0.7604 - accuracy: 0.6777 - val_loss: 1.2264 - val_accuracy: 0.5128
Epoch 60/1000
273/273 [==============================] - 0s 73us/step - loss: 0.8377 - accuracy: 0.6777 - val_loss: 1.0093 - val_accuracy: 0.4957
Epoch 61/1000
273/273 [==============================] - 0s 63us/step - loss: 0.7422 - accuracy: 0.7106 - val_loss: 1.1088 - val_accuracy: 0.5726
Epoch 62/1000
273/273 [==============================] - 0s 60us/step - loss: 0.8006 - accuracy: 0.6996 - val_loss: 0.9370 - val_accuracy: 0.5897
Epoch 63/1000
273/273 [==============================] - 0s 57us/step - loss: 0.7143 - accuracy: 0.7033 - val_loss: 0.9487 -

273/273 [==============================] - 0s 63us/step - loss: 0.7007 - accuracy: 0.7802 - val_loss: 1.1436 - val_accuracy: 0.6239
Epoch 114/1000
273/273 [==============================] - 0s 59us/step - loss: 0.6329 - accuracy: 0.7619 - val_loss: 1.1045 - val_accuracy: 0.5299
Epoch 115/1000
273/273 [==============================] - 0s 63us/step - loss: 0.6634 - accuracy: 0.7289 - val_loss: 1.0269 - val_accuracy: 0.5726
Epoch 116/1000
273/273 [==============================] - 0s 58us/step - loss: 0.5973 - accuracy: 0.7509 - val_loss: 0.9403 - val_accuracy: 0.5726
Epoch 117/1000
273/273 [==============================] - 0s 61us/step - loss: 0.5606 - accuracy: 0.7656 - val_loss: 0.9443 - val_accuracy: 0.5983
Epoch 118/1000
273/273 [==============================] - 0s 62us/step - loss: 0.5637 - accuracy: 0.7692 - val_loss: 0.9373 - val_accuracy: 0.6239
Epoch 119/1000
273/273 [==============================] - 0s 67us/step - loss: 0.5550 - accuracy: 0.7692 - val_loss: 0.9448 - val_acc

Epoch 169/1000
273/273 [==============================] - 0s 60us/step - loss: 1.4254 - accuracy: 0.7216 - val_loss: 2.1650 - val_accuracy: 0.5299
Epoch 170/1000
273/273 [==============================] - 0s 55us/step - loss: 1.1147 - accuracy: 0.7253 - val_loss: 1.5039 - val_accuracy: 0.5897
Epoch 171/1000
273/273 [==============================] - 0s 66us/step - loss: 0.7918 - accuracy: 0.7106 - val_loss: 1.2811 - val_accuracy: 0.5385
Epoch 172/1000
273/273 [==============================] - 0s 63us/step - loss: 1.1560 - accuracy: 0.7289 - val_loss: 2.5222 - val_accuracy: 0.5726
Epoch 173/1000
273/273 [==============================] - 0s 61us/step - loss: 1.2526 - accuracy: 0.7253 - val_loss: 1.5091 - val_accuracy: 0.5812
Epoch 174/1000
273/273 [==============================] - 0s 63us/step - loss: 0.7622 - accuracy: 0.7399 - val_loss: 1.2926 - val_accuracy: 0.5043
Epoch 175/1000
273/273 [==============================] - 0s 71us/step - loss: 0.7104 - accuracy: 0.6996 - val_loss: 1

Epoch 225/1000
273/273 [==============================] - 0s 58us/step - loss: 0.6786 - accuracy: 0.8132 - val_loss: 1.1158 - val_accuracy: 0.6325
Epoch 226/1000
273/273 [==============================] - 0s 57us/step - loss: 0.5173 - accuracy: 0.7802 - val_loss: 1.0330 - val_accuracy: 0.5983
Epoch 227/1000
273/273 [==============================] - 0s 62us/step - loss: 0.4747 - accuracy: 0.8022 - val_loss: 1.0111 - val_accuracy: 0.6410
Epoch 228/1000
273/273 [==============================] - 0s 60us/step - loss: 0.4683 - accuracy: 0.7985 - val_loss: 1.0026 - val_accuracy: 0.6496
Epoch 229/1000
273/273 [==============================] - 0s 60us/step - loss: 0.4799 - accuracy: 0.8132 - val_loss: 0.9897 - val_accuracy: 0.6325
Epoch 230/1000
273/273 [==============================] - 0s 60us/step - loss: 0.4407 - accuracy: 0.8059 - val_loss: 0.9551 - val_accuracy: 0.6752
Epoch 231/1000
273/273 [==============================] - 0s 54us/step - loss: 0.4377 - accuracy: 0.8095 - val_loss: 0

...

Epoch 729/1000
273/273 [==============================] - 0s 57us/step - loss: 0.3296 - accuracy: 0.8571 - val_loss: 1.1929 - val_accuracy: 0.7179
Epoch 730/1000
273/273 [==============================] - 0s 49us/step - loss: 0.3292 - accuracy: 0.8571 - val_loss: 1.1893 - val_accuracy: 0.7179
Epoch 731/1000
273/273 [==============================] - 0s 63us/step - loss: 0.3271 - accuracy: 0.8571 - val_loss: 1.2030 - val_accuracy: 0.7009
Epoch 732/1000
273/273 [==============================] - 0s 76us/step - loss: 0.3371 - accuracy: 0.8462 - val_loss: 1.2251 - val_accuracy: 0.7179
Epoch 733/1000
273/273 [==============================] - 0s 88us/step - loss: 0.3305 - accuracy: 0.8718 - val_loss: 1.2423 - val_accuracy: 0.7179
Epoch 734/1000
273/273 [==============================] - 0s 68us/step - loss: 0.3280 - accuracy: 0.8681 - val_loss: 1.2357 - val_accuracy: 0.7265
Epoch 735/1000
273/273 [==============================] - 0s 77us/step - loss: 0.3255 - accuracy: 0.8681 - val_loss: 1

Epoch 785/1000
273/273 [==============================] - 0s 69us/step - loss: 0.3208 - accuracy: 0.8608 - val_loss: 1.2806 - val_accuracy: 0.7094
Epoch 786/1000
273/273 [==============================] - 0s 61us/step - loss: 0.3164 - accuracy: 0.8645 - val_loss: 1.2581 - val_accuracy: 0.7179
Epoch 787/1000
273/273 [==============================] - 0s 65us/step - loss: 0.3117 - accuracy: 0.8645 - val_loss: 1.2525 - val_accuracy: 0.7094
Epoch 788/1000
273/273 [==============================] - 0s 47us/step - loss: 0.3185 - accuracy: 0.8681 - val_loss: 1.2691 - val_accuracy: 0.7179
Epoch 789/1000
273/273 [==============================] - 0s 68us/step - loss: 0.3199 - accuracy: 0.8681 - val_loss: 1.2705 - val_accuracy: 0.7265
Epoch 790/1000
273/273 [==============================] - 0s 64us/step - loss: 0.3154 - accuracy: 0.8681 - val_loss: 1.2648 - val_accuracy: 0.7094
Epoch 791/1000
273/273 [==============================] - 0s 58us/step - loss: 0.3261 - accuracy: 0.8608 - val_loss: 1

Epoch 841/1000
273/273 [==============================] - 0s 58us/step - loss: 0.3194 - accuracy: 0.8681 - val_loss: 1.3005 - val_accuracy: 0.7094
Epoch 842/1000
273/273 [==============================] - 0s 65us/step - loss: 0.3142 - accuracy: 0.8718 - val_loss: 1.3132 - val_accuracy: 0.7179
Epoch 843/1000
273/273 [==============================] - 0s 64us/step - loss: 0.3175 - accuracy: 0.8571 - val_loss: 1.3118 - val_accuracy: 0.6667
Epoch 844/1000
273/273 [==============================] - 0s 55us/step - loss: 0.3204 - accuracy: 0.8645 - val_loss: 1.3112 - val_accuracy: 0.6581
Epoch 845/1000
273/273 [==============================] - 0s 56us/step - loss: 0.3054 - accuracy: 0.8645 - val_loss: 1.2803 - val_accuracy: 0.6581
Epoch 846/1000
273/273 [==============================] - 0s 53us/step - loss: 0.3046 - accuracy: 0.8571 - val_loss: 1.2769 - val_accuracy: 0.7009
Epoch 847/1000
273/273 [==============================] - 0s 58us/step - loss: 0.3193 - accuracy: 0.8571 - val_loss: 1

Epoch 897/1000
273/273 [==============================] - 0s 59us/step - loss: 0.7025 - accuracy: 0.8278 - val_loss: 1.4244 - val_accuracy: 0.6838
Epoch 898/1000
273/273 [==============================] - 0s 56us/step - loss: 0.5025 - accuracy: 0.7839 - val_loss: 1.4659 - val_accuracy: 0.6410
Epoch 899/1000
273/273 [==============================] - 0s 63us/step - loss: 0.5948 - accuracy: 0.8205 - val_loss: 2.1784 - val_accuracy: 0.6496
Epoch 900/1000
273/273 [==============================] - 0s 61us/step - loss: 0.7984 - accuracy: 0.8242 - val_loss: 1.7494 - val_accuracy: 0.6496
Epoch 901/1000
273/273 [==============================] - 0s 59us/step - loss: 0.5070 - accuracy: 0.8168 - val_loss: 2.1774 - val_accuracy: 0.5812
Epoch 902/1000
273/273 [==============================] - 0s 54us/step - loss: 0.7495 - accuracy: 0.7985 - val_loss: 1.7658 - val_accuracy: 0.6068
Epoch 903/1000
273/273 [==============================] - 0s 59us/step - loss: 0.5751 - accuracy: 0.8242 - val_loss: 2

Epoch 953/1000
273/273 [==============================] - 0s 52us/step - loss: 0.3694 - accuracy: 0.8462 - val_loss: 1.3692 - val_accuracy: 0.7094
Epoch 954/1000
273/273 [==============================] - 0s 56us/step - loss: 0.3141 - accuracy: 0.8608 - val_loss: 1.3833 - val_accuracy: 0.7094
Epoch 955/1000
273/273 [==============================] - 0s 65us/step - loss: 0.3275 - accuracy: 0.8571 - val_loss: 1.4156 - val_accuracy: 0.6581
Epoch 956/1000
273/273 [==============================] - 0s 57us/step - loss: 0.3211 - accuracy: 0.8535 - val_loss: 1.4193 - val_accuracy: 0.6667
Epoch 957/1000
273/273 [==============================] - 0s 59us/step - loss: 0.3138 - accuracy: 0.8645 - val_loss: 1.3752 - val_accuracy: 0.7094
Epoch 958/1000
273/273 [==============================] - 0s 50us/step - loss: 0.3094 - accuracy: 0.8645 - val_loss: 1.3369 - val_accuracy: 0.7179
Epoch 959/1000
273/273 [==============================] - 0s 60us/step - loss: 0.3286 - accuracy: 0.8608 - val_loss: 1

In [75]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

117/117 [==============================] - 0s 97us/step
over-sampling test accuracy: 72.65%


In [25]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [26]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 71 samples, validate on 31 samples
Epoch 1/100
71/71 [==============================] - 0s 1ms/step - loss: 12.3466 - accuracy: 0.2394 - val_loss: 10.8846 - val_accuracy: 0.4516
Epoch 2/100
71/71 [==============================] - 0s 119us/step - loss: 10.2216 - accuracy: 0.5493 - val_loss: 8.8981 - val_accuracy: 0.4839
Epoch 3/100
71/71 [==============================] - 0s 176us/step - loss: 8.5359 - accuracy: 0.5634 - val_loss: 7.0163 - val_accuracy: 0.4839
Epoch 4/100
71/71 [==============================] - 0s 179us/step - loss: 6.6900 - accuracy: 0.5634 - val_loss: 5.2855 - val_accuracy: 0.4839
Epoch 5/100
71/71 [==============================] - 0s 296us/step - loss: 5.0221 - accuracy: 0.5634 - val_loss: 3.5704 - val_accuracy: 0.5161
Epoch 6/100
71/71 [==============================] - 0s 241us/step - loss: 3.7939 - accuracy: 0.5915 - val_loss: 2.0697 - val_accuracy: 0.6129
Epoch 7/100
71/71 [==============================] - 0s 193us/step - loss: 3.0774 - accuracy: 0.6

71/71 [==============================] - 0s 216us/step - loss: 0.6081 - accuracy: 0.8592 - val_loss: 0.4328 - val_accuracy: 0.8710
Epoch 58/100
71/71 [==============================] - 0s 266us/step - loss: 0.3684 - accuracy: 0.8732 - val_loss: 0.4665 - val_accuracy: 0.8387
Epoch 59/100
71/71 [==============================] - 0s 281us/step - loss: 0.4822 - accuracy: 0.8451 - val_loss: 0.4998 - val_accuracy: 0.8387
Epoch 60/100
71/71 [==============================] - 0s 220us/step - loss: 0.5382 - accuracy: 0.8592 - val_loss: 0.4675 - val_accuracy: 0.8710
Epoch 61/100
71/71 [==============================] - 0s 196us/step - loss: 0.4605 - accuracy: 0.8592 - val_loss: 0.3940 - val_accuracy: 0.9032
Epoch 62/100
71/71 [==============================] - 0s 191us/step - loss: 0.3818 - accuracy: 0.8732 - val_loss: 0.4106 - val_accuracy: 0.8387
Epoch 63/100
71/71 [==============================] - 0s 153us/step - loss: 0.3506 - accuracy: 0.8732 - val_loss: 0.5579 - val_accuracy: 0.8387
Epoch

In [28]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

31/31 [==============================] - 0s 122us/step
combination test accuracy: 83.87%


In [30]:
#### improve neural network on combination data
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [31]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 71 samples, validate on 31 samples
Epoch 1/1000
71/71 [==============================] - 0s 2ms/step - loss: 13.0515 - accuracy: 0.5352 - val_loss: 9.1546 - val_accuracy: 0.6452
Epoch 2/1000
71/71 [==============================] - 0s 144us/step - loss: 11.1963 - accuracy: 0.6479 - val_loss: 8.3031 - val_accuracy: 0.5161
Epoch 3/1000
71/71 [==============================] - 0s 224us/step - loss: 9.5139 - accuracy: 0.6056 - val_loss: 6.4922 - val_accuracy: 0.4839
Epoch 4/1000
71/71 [==============================] - 0s 219us/step - loss: 7.2126 - accuracy: 0.6479 - val_loss: 4.7171 - val_accuracy: 0.6774
Epoch 5/1000
71/71 [==============================] - 0s 219us/step - loss: 5.5371 - accuracy: 0.6901 - val_loss: 3.4875 - val_accuracy: 0.6774
Epoch 6/1000
71/71 [==============================] - 0s 253us/step - loss: 4.2053 - accuracy: 0.6901 - val_loss: 2.4575 - val_accuracy: 0.6129
Epoch 7/1000
71/71 [==============================] - 0s 228us/step - loss: 2.6320 - accurac

71/71 [==============================] - 0s 203us/step - loss: 0.6101 - accuracy: 0.8592 - val_loss: 0.8162 - val_accuracy: 0.8387
Epoch 58/1000
71/71 [==============================] - 0s 172us/step - loss: 0.5840 - accuracy: 0.8592 - val_loss: 0.6016 - val_accuracy: 0.8710
Epoch 59/1000
71/71 [==============================] - 0s 176us/step - loss: 0.4536 - accuracy: 0.8451 - val_loss: 0.4617 - val_accuracy: 0.8387
Epoch 60/1000
71/71 [==============================] - 0s 215us/step - loss: 0.4263 - accuracy: 0.8732 - val_loss: 0.5366 - val_accuracy: 0.8387
Epoch 61/1000
71/71 [==============================] - 0s 230us/step - loss: 0.5761 - accuracy: 0.8451 - val_loss: 0.5286 - val_accuracy: 0.8710
Epoch 62/1000
71/71 [==============================] - 0s 183us/step - loss: 0.5725 - accuracy: 0.8592 - val_loss: 0.4260 - val_accuracy: 0.8710
Epoch 63/1000
71/71 [==============================] - 0s 208us/step - loss: 0.3434 - accuracy: 0.8592 - val_loss: 0.7150 - val_accuracy: 0.8065

71/71 [==============================] - 0s 198us/step - loss: 0.2247 - accuracy: 0.9437 - val_loss: 0.3063 - val_accuracy: 0.9355
Epoch 114/1000
71/71 [==============================] - 0s 181us/step - loss: 0.3156 - accuracy: 0.9155 - val_loss: 0.3594 - val_accuracy: 0.9032
Epoch 115/1000
71/71 [==============================] - 0s 181us/step - loss: 0.2774 - accuracy: 0.9155 - val_loss: 0.3760 - val_accuracy: 0.9032
Epoch 116/1000
71/71 [==============================] - 0s 150us/step - loss: 0.2453 - accuracy: 0.9296 - val_loss: 0.2673 - val_accuracy: 0.9355
Epoch 117/1000
71/71 [==============================] - 0s 203us/step - loss: 0.2581 - accuracy: 0.8873 - val_loss: 0.2843 - val_accuracy: 0.9677
Epoch 118/1000
71/71 [==============================] - 0s 238us/step - loss: 0.3031 - accuracy: 0.9296 - val_loss: 0.3142 - val_accuracy: 0.9677
Epoch 119/1000
71/71 [==============================] - 0s 178us/step - loss: 0.3593 - accuracy: 0.9296 - val_loss: 0.2407 - val_accuracy: 

71/71 [==============================] - 0s 185us/step - loss: 0.1957 - accuracy: 0.9577 - val_loss: 0.2382 - val_accuracy: 0.9677
Epoch 170/1000
71/71 [==============================] - 0s 199us/step - loss: 0.1909 - accuracy: 0.9577 - val_loss: 0.2342 - val_accuracy: 0.9355
Epoch 171/1000
71/71 [==============================] - 0s 208us/step - loss: 0.2623 - accuracy: 0.9296 - val_loss: 0.2481 - val_accuracy: 0.9355
Epoch 172/1000
71/71 [==============================] - 0s 223us/step - loss: 0.2699 - accuracy: 0.9296 - val_loss: 0.1554 - val_accuracy: 0.9677
Epoch 173/1000
71/71 [==============================] - 0s 176us/step - loss: 0.1378 - accuracy: 0.9718 - val_loss: 0.1612 - val_accuracy: 0.9677
Epoch 174/1000
71/71 [==============================] - 0s 185us/step - loss: 0.1812 - accuracy: 0.9437 - val_loss: 0.3005 - val_accuracy: 0.9355
Epoch 175/1000
71/71 [==============================] - 0s 190us/step - loss: 0.3412 - accuracy: 0.9296 - val_loss: 0.2508 - val_accuracy: 

71/71 [==============================] - 0s 203us/step - loss: 0.1430 - accuracy: 0.9437 - val_loss: 0.1037 - val_accuracy: 1.0000
Epoch 226/1000
71/71 [==============================] - 0s 197us/step - loss: 0.1161 - accuracy: 0.9577 - val_loss: 0.1082 - val_accuracy: 0.9677
Epoch 227/1000
71/71 [==============================] - 0s 200us/step - loss: 0.1615 - accuracy: 0.9577 - val_loss: 0.1220 - val_accuracy: 0.9677
Epoch 228/1000
71/71 [==============================] - 0s 205us/step - loss: 0.1311 - accuracy: 0.9437 - val_loss: 0.1360 - val_accuracy: 0.9355
Epoch 229/1000
71/71 [==============================] - 0s 235us/step - loss: 0.1341 - accuracy: 0.9577 - val_loss: 0.0997 - val_accuracy: 0.9677
Epoch 230/1000
71/71 [==============================] - 0s 210us/step - loss: 0.1003 - accuracy: 0.9577 - val_loss: 0.0985 - val_accuracy: 1.0000
Epoch 231/1000
71/71 [==============================] - 0s 181us/step - loss: 0.0885 - accuracy: 0.9577 - val_loss: 0.1879 - val_accuracy: 

...

71/71 [==============================] - 0s 132us/step - loss: 0.0360 - accuracy: 0.9859 - val_loss: 0.0603 - val_accuracy: 1.0000
Epoch 730/1000
71/71 [==============================] - 0s 138us/step - loss: 0.0480 - accuracy: 0.9718 - val_loss: 0.0599 - val_accuracy: 1.0000
Epoch 731/1000
71/71 [==============================] - 0s 146us/step - loss: 0.0389 - accuracy: 0.9859 - val_loss: 0.0525 - val_accuracy: 1.0000
Epoch 732/1000
71/71 [==============================] - 0s 193us/step - loss: 0.0369 - accuracy: 0.9859 - val_loss: 0.0501 - val_accuracy: 1.0000
Epoch 733/1000
71/71 [==============================] - 0s 163us/step - loss: 0.0404 - accuracy: 0.9859 - val_loss: 0.0498 - val_accuracy: 1.0000
Epoch 734/1000
71/71 [==============================] - 0s 126us/step - loss: 0.0371 - accuracy: 0.9859 - val_loss: 0.0518 - val_accuracy: 1.0000
Epoch 735/1000
71/71 [==============================] - 0s 183us/step - loss: 0.0336 - accuracy: 0.9859 - val_loss: 0.0537 - val_accuracy: 

Epoch 785/1000
71/71 [==============================] - 0s 165us/step - loss: 0.0284 - accuracy: 0.9859 - val_loss: 0.0397 - val_accuracy: 1.0000
Epoch 786/1000
71/71 [==============================] - 0s 273us/step - loss: 0.0319 - accuracy: 0.9859 - val_loss: 0.0388 - val_accuracy: 1.0000
Epoch 787/1000
71/71 [==============================] - 0s 179us/step - loss: 0.0292 - accuracy: 0.9859 - val_loss: 0.0582 - val_accuracy: 1.0000
Epoch 788/1000
71/71 [==============================] - 0s 195us/step - loss: 0.0554 - accuracy: 0.9718 - val_loss: 0.0355 - val_accuracy: 1.0000
Epoch 789/1000
71/71 [==============================] - 0s 180us/step - loss: 0.0271 - accuracy: 1.0000 - val_loss: 0.0321 - val_accuracy: 1.0000
Epoch 790/1000
71/71 [==============================] - 0s 144us/step - loss: 0.0552 - accuracy: 0.9718 - val_loss: 0.0320 - val_accuracy: 1.0000
Epoch 791/1000
71/71 [==============================] - 0s 172us/step - loss: 0.0483 - accuracy: 0.9859 - val_loss: 0.0340 -

71/71 [==============================] - 0s 177us/step - loss: 0.1545 - accuracy: 0.9577 - val_loss: 0.0591 - val_accuracy: 0.9677
Epoch 842/1000
71/71 [==============================] - 0s 187us/step - loss: 0.0664 - accuracy: 0.9718 - val_loss: 0.1624 - val_accuracy: 0.9355
Epoch 843/1000
71/71 [==============================] - 0s 172us/step - loss: 0.1705 - accuracy: 0.9437 - val_loss: 0.0623 - val_accuracy: 0.9677
Epoch 844/1000
71/71 [==============================] - 0s 194us/step - loss: 0.0764 - accuracy: 0.9859 - val_loss: 0.1941 - val_accuracy: 0.9355
Epoch 845/1000
71/71 [==============================] - 0s 164us/step - loss: 0.3337 - accuracy: 0.9437 - val_loss: 0.3115 - val_accuracy: 0.9677
Epoch 846/1000
71/71 [==============================] - 0s 149us/step - loss: 0.4911 - accuracy: 0.9437 - val_loss: 0.2395 - val_accuracy: 0.9355
Epoch 847/1000
71/71 [==============================] - 0s 142us/step - loss: 0.3166 - accuracy: 0.9577 - val_loss: 0.0661 - val_accuracy: 

71/71 [==============================] - 0s 178us/step - loss: 0.0299 - accuracy: 1.0000 - val_loss: 0.0348 - val_accuracy: 0.9677
Epoch 898/1000
71/71 [==============================] - 0s 203us/step - loss: 0.0206 - accuracy: 1.0000 - val_loss: 0.0409 - val_accuracy: 0.9677
Epoch 899/1000
71/71 [==============================] - 0s 198us/step - loss: 0.0321 - accuracy: 0.9859 - val_loss: 0.0366 - val_accuracy: 0.9677
Epoch 900/1000
71/71 [==============================] - 0s 153us/step - loss: 0.0227 - accuracy: 1.0000 - val_loss: 0.0333 - val_accuracy: 1.0000
Epoch 901/1000
71/71 [==============================] - 0s 178us/step - loss: 0.0248 - accuracy: 1.0000 - val_loss: 0.0322 - val_accuracy: 1.0000
Epoch 902/1000
71/71 [==============================] - 0s 137us/step - loss: 0.0252 - accuracy: 1.0000 - val_loss: 0.0332 - val_accuracy: 1.0000
Epoch 903/1000
71/71 [==============================] - 0s 168us/step - loss: 0.0211 - accuracy: 1.0000 - val_loss: 0.0380 - val_accuracy: 

71/71 [==============================] - 0s 174us/step - loss: 0.0314 - accuracy: 0.9859 - val_loss: 0.0411 - val_accuracy: 1.0000
Epoch 954/1000
71/71 [==============================] - 0s 169us/step - loss: 0.1541 - accuracy: 0.9577 - val_loss: 0.0408 - val_accuracy: 1.0000
Epoch 955/1000
71/71 [==============================] - 0s 198us/step - loss: 0.0911 - accuracy: 0.9859 - val_loss: 0.0402 - val_accuracy: 0.9677
Epoch 956/1000
71/71 [==============================] - 0s 150us/step - loss: 0.0606 - accuracy: 0.9577 - val_loss: 0.0415 - val_accuracy: 0.9677
Epoch 957/1000
71/71 [==============================] - 0s 166us/step - loss: 0.0225 - accuracy: 1.0000 - val_loss: 0.0531 - val_accuracy: 0.9677
Epoch 958/1000
71/71 [==============================] - ETA: 0s - loss: 0.2920 - accuracy: 0.93 - 0s 154us/step - loss: 0.1628 - accuracy: 0.9577 - val_loss: 0.1060 - val_accuracy: 0.9355
Epoch 959/1000
71/71 [==============================] - 0s 134us/step - loss: 0.2329 - accuracy: 

In [34]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

31/31 [==============================] - 0s 107us/step
combination test accuracy: 96.77%


In [47]:
#### add dropout
model1_comb3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [48]:
model1_comb3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [49]:
model1_comb3.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 71 samples, validate on 31 samples
Epoch 1/1000
71/71 [==============================] - 0s 3ms/step - loss: 29.1807 - accuracy: 0.5211 - val_loss: 26.3565 - val_accuracy: 0.6452
Epoch 2/1000
71/71 [==============================] - 0s 156us/step - loss: 29.7512 - accuracy: 0.5070 - val_loss: 25.7730 - val_accuracy: 0.6452
Epoch 3/1000
71/71 [==============================] - 0s 279us/step - loss: 29.6204 - accuracy: 0.4930 - val_loss: 25.2010 - val_accuracy: 0.6452
Epoch 4/1000
71/71 [==============================] - 0s 251us/step - loss: 27.9975 - accuracy: 0.5915 - val_loss: 24.4771 - val_accuracy: 0.6452
Epoch 5/1000
71/71 [==============================] - 0s 241us/step - loss: 25.2000 - accuracy: 0.5352 - val_loss: 23.7893 - val_accuracy: 0.6452
Epoch 6/1000
71/71 [==============================] - 0s 262us/step - loss: 27.2935 - accuracy: 0.6056 - val_loss: 23.1065 - val_accuracy: 0.6452
Epoch 7/1000
71/71 [==============================] - 0s 243us/step - loss: 26.637

71/71 [==============================] - 0s 285us/step - loss: 10.9767 - accuracy: 0.5634 - val_loss: 9.0505 - val_accuracy: 0.6774
Epoch 57/1000
71/71 [==============================] - 0s 280us/step - loss: 9.4304 - accuracy: 0.7042 - val_loss: 8.8218 - val_accuracy: 0.7097
Epoch 58/1000
71/71 [==============================] - 0s 289us/step - loss: 11.1234 - accuracy: 0.5915 - val_loss: 8.5982 - val_accuracy: 0.7097
Epoch 59/1000
71/71 [==============================] - 0s 179us/step - loss: 11.9013 - accuracy: 0.5775 - val_loss: 8.3766 - val_accuracy: 0.7097
Epoch 60/1000
71/71 [==============================] - 0s 164us/step - loss: 11.2929 - accuracy: 0.5915 - val_loss: 8.2024 - val_accuracy: 0.7097
Epoch 61/1000
71/71 [==============================] - 0s 209us/step - loss: 10.4730 - accuracy: 0.6761 - val_loss: 8.0485 - val_accuracy: 0.7097
Epoch 62/1000
71/71 [==============================] - 0s 203us/step - loss: 11.2428 - accuracy: 0.5915 - val_loss: 7.8799 - val_accuracy: 

71/71 [==============================] - 0s 166us/step - loss: 4.4915 - accuracy: 0.6479 - val_loss: 2.5969 - val_accuracy: 0.7097
Epoch 113/1000
71/71 [==============================] - 0s 162us/step - loss: 3.9388 - accuracy: 0.6620 - val_loss: 2.5446 - val_accuracy: 0.7097
Epoch 114/1000
71/71 [==============================] - 0s 188us/step - loss: 3.5435 - accuracy: 0.7324 - val_loss: 2.5176 - val_accuracy: 0.7097
Epoch 115/1000
71/71 [==============================] - 0s 181us/step - loss: 3.8806 - accuracy: 0.6197 - val_loss: 2.4751 - val_accuracy: 0.7097
Epoch 116/1000
71/71 [==============================] - 0s 209us/step - loss: 4.5239 - accuracy: 0.6338 - val_loss: 2.3962 - val_accuracy: 0.7097
Epoch 117/1000
71/71 [==============================] - 0s 196us/step - loss: 4.1846 - accuracy: 0.5915 - val_loss: 2.2884 - val_accuracy: 0.7742
Epoch 118/1000
71/71 [==============================] - 0s 167us/step - loss: 3.0360 - accuracy: 0.7183 - val_loss: 2.2145 - val_accuracy: 

71/71 [==============================] - 0s 136us/step - loss: 2.0630 - accuracy: 0.6761 - val_loss: 0.5600 - val_accuracy: 0.8710
Epoch 169/1000
71/71 [==============================] - 0s 146us/step - loss: 1.9370 - accuracy: 0.7746 - val_loss: 0.5330 - val_accuracy: 0.9032
Epoch 170/1000
71/71 [==============================] - 0s 133us/step - loss: 2.0150 - accuracy: 0.6620 - val_loss: 0.5176 - val_accuracy: 0.9032
Epoch 171/1000
71/71 [==============================] - 0s 154us/step - loss: 1.6042 - accuracy: 0.7465 - val_loss: 0.5096 - val_accuracy: 0.9355
Epoch 172/1000
71/71 [==============================] - 0s 158us/step - loss: 2.1484 - accuracy: 0.7324 - val_loss: 0.5092 - val_accuracy: 0.9355
Epoch 173/1000
71/71 [==============================] - 0s 157us/step - loss: 2.4733 - accuracy: 0.6901 - val_loss: 0.5476 - val_accuracy: 0.9032
Epoch 174/1000
71/71 [==============================] - 0s 145us/step - loss: 1.3638 - accuracy: 0.7606 - val_loss: 0.5334 - val_accuracy: 

71/71 [==============================] - 0s 177us/step - loss: 1.4868 - accuracy: 0.7746 - val_loss: 0.4207 - val_accuracy: 0.9032
Epoch 225/1000
71/71 [==============================] - 0s 173us/step - loss: 1.7464 - accuracy: 0.6620 - val_loss: 0.4208 - val_accuracy: 0.9032
Epoch 226/1000
71/71 [==============================] - 0s 183us/step - loss: 1.9067 - accuracy: 0.7606 - val_loss: 0.4202 - val_accuracy: 0.9032
Epoch 227/1000
71/71 [==============================] - 0s 141us/step - loss: 2.1162 - accuracy: 0.6901 - val_loss: 0.4257 - val_accuracy: 0.9032
Epoch 228/1000
71/71 [==============================] - 0s 173us/step - loss: 1.0236 - accuracy: 0.7887 - val_loss: 0.4314 - val_accuracy: 0.9032
Epoch 229/1000
71/71 [==============================] - 0s 187us/step - loss: 0.9793 - accuracy: 0.7887 - val_loss: 0.4346 - val_accuracy: 0.9032
Epoch 230/1000
71/71 [==============================] - 0s 173us/step - loss: 1.8016 - accuracy: 0.7324 - val_loss: 0.4269 - val_accuracy: 

...

71/71 [==============================] - 0s 199us/step - loss: 0.8699 - accuracy: 0.7606 - val_loss: 0.4394 - val_accuracy: 0.8710
Epoch 729/1000
71/71 [==============================] - 0s 184us/step - loss: 1.2324 - accuracy: 0.6761 - val_loss: 0.4116 - val_accuracy: 0.8710
Epoch 730/1000
71/71 [==============================] - 0s 230us/step - loss: 0.6441 - accuracy: 0.7746 - val_loss: 0.3853 - val_accuracy: 0.9032
Epoch 731/1000
71/71 [==============================] - 0s 171us/step - loss: 0.5777 - accuracy: 0.8592 - val_loss: 0.3727 - val_accuracy: 0.9032
Epoch 732/1000
71/71 [==============================] - 0s 177us/step - loss: 1.1446 - accuracy: 0.7183 - val_loss: 0.3671 - val_accuracy: 0.9032
Epoch 733/1000
71/71 [==============================] - 0s 165us/step - loss: 1.1100 - accuracy: 0.7606 - val_loss: 0.3645 - val_accuracy: 0.9032
Epoch 734/1000
71/71 [==============================] - 0s 212us/step - loss: 0.9857 - accuracy: 0.7042 - val_loss: 0.3630 - val_accuracy: 

71/71 [==============================] - 0s 174us/step - loss: 1.1829 - accuracy: 0.7183 - val_loss: 0.2014 - val_accuracy: 0.9677
Epoch 785/1000
71/71 [==============================] - 0s 155us/step - loss: 0.6385 - accuracy: 0.7465 - val_loss: 0.2028 - val_accuracy: 0.9677
Epoch 786/1000
71/71 [==============================] - 0s 166us/step - loss: 0.5802 - accuracy: 0.7887 - val_loss: 0.1973 - val_accuracy: 0.9677
Epoch 787/1000
71/71 [==============================] - 0s 213us/step - loss: 0.8708 - accuracy: 0.7465 - val_loss: 0.1947 - val_accuracy: 0.9677
Epoch 788/1000
71/71 [==============================] - 0s 183us/step - loss: 0.3357 - accuracy: 0.8310 - val_loss: 0.1996 - val_accuracy: 0.9677
Epoch 789/1000
71/71 [==============================] - 0s 188us/step - loss: 0.5192 - accuracy: 0.7183 - val_loss: 0.2199 - val_accuracy: 0.9677
Epoch 790/1000
71/71 [==============================] - 0s 190us/step - loss: 0.5535 - accuracy: 0.7324 - val_loss: 0.2055 - val_accuracy: 

Epoch 840/1000
71/71 [==============================] - 0s 153us/step - loss: 0.5464 - accuracy: 0.7465 - val_loss: 0.2121 - val_accuracy: 0.9677
Epoch 841/1000
71/71 [==============================] - 0s 206us/step - loss: 0.7706 - accuracy: 0.8310 - val_loss: 0.2136 - val_accuracy: 0.9677
Epoch 842/1000
71/71 [==============================] - 0s 156us/step - loss: 0.8305 - accuracy: 0.7606 - val_loss: 0.2120 - val_accuracy: 0.9677
Epoch 843/1000
71/71 [==============================] - 0s 191us/step - loss: 0.8715 - accuracy: 0.8169 - val_loss: 0.2106 - val_accuracy: 0.9677
Epoch 844/1000
71/71 [==============================] - 0s 170us/step - loss: 1.2281 - accuracy: 0.6197 - val_loss: 0.2100 - val_accuracy: 0.9677
Epoch 845/1000
71/71 [==============================] - 0s 170us/step - loss: 0.5822 - accuracy: 0.8169 - val_loss: 0.2093 - val_accuracy: 0.9677
Epoch 846/1000
71/71 [==============================] - 0s 157us/step - loss: 0.7191 - accuracy: 0.8732 - val_loss: 0.2086 -

71/71 [==============================] - 0s 178us/step - loss: 0.4541 - accuracy: 0.7606 - val_loss: 0.1927 - val_accuracy: 0.9677
Epoch 897/1000
71/71 [==============================] - 0s 192us/step - loss: 0.5883 - accuracy: 0.8169 - val_loss: 0.1926 - val_accuracy: 0.9677
Epoch 898/1000
71/71 [==============================] - 0s 209us/step - loss: 0.9733 - accuracy: 0.8028 - val_loss: 0.1924 - val_accuracy: 0.9677
Epoch 899/1000
71/71 [==============================] - 0s 162us/step - loss: 0.8020 - accuracy: 0.7887 - val_loss: 0.1920 - val_accuracy: 0.9677
Epoch 900/1000
71/71 [==============================] - 0s 185us/step - loss: 0.9065 - accuracy: 0.7606 - val_loss: 0.1926 - val_accuracy: 0.9677
Epoch 901/1000
71/71 [==============================] - 0s 181us/step - loss: 1.2743 - accuracy: 0.7465 - val_loss: 0.1948 - val_accuracy: 0.9677
Epoch 902/1000
71/71 [==============================] - 0s 169us/step - loss: 0.5300 - accuracy: 0.8451 - val_loss: 0.1947 - val_accuracy: 

71/71 [==============================] - 0s 175us/step - loss: 0.8268 - accuracy: 0.7465 - val_loss: 0.1881 - val_accuracy: 0.9677
Epoch 953/1000
71/71 [==============================] - 0s 172us/step - loss: 0.7128 - accuracy: 0.8732 - val_loss: 0.1880 - val_accuracy: 0.9677
Epoch 954/1000
71/71 [==============================] - 0s 174us/step - loss: 0.5240 - accuracy: 0.8310 - val_loss: 0.1876 - val_accuracy: 0.9677
Epoch 955/1000
71/71 [==============================] - 0s 186us/step - loss: 0.6843 - accuracy: 0.7324 - val_loss: 0.1886 - val_accuracy: 0.9677
Epoch 956/1000
71/71 [==============================] - 0s 181us/step - loss: 0.8222 - accuracy: 0.7746 - val_loss: 0.1890 - val_accuracy: 0.9677
Epoch 957/1000
71/71 [==============================] - 0s 158us/step - loss: 0.6131 - accuracy: 0.7606 - val_loss: 0.1889 - val_accuracy: 0.9677
Epoch 958/1000
71/71 [==============================] - 0s 165us/step - loss: 0.3433 - accuracy: 0.8310 - val_loss: 0.1883 - val_accuracy: 

In [50]:
acc_test_comb3 = model1_comb3.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

31/31 [==============================] - 0s 74us/step
combination test accuracy: 96.77%


In [51]:
############## Random Forest ##############

In [52]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [53]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 69.23%


In [58]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [59]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [56]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.64912281 0.66666667 0.66666667 0.7037037  0.75925926]
0.6890838206627681


In [57]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.93333333 0.86666667 0.8        0.85714286 1.        ]
0.8914285714285715
